In [1]:
import pandas as pd
from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from sklearn.model_selection import train_test_split

tqdm.pandas()

In [2]:
from transformers import BertTokenizer, BertModel
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased", torchscript=True)
model.to(device)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

In [3]:
print(device)

cuda


In [4]:
df = pd.read_csv("women-clothing-accessories.3-class.balanced.csv", delimiter='\t')
df = df[df['sentiment'] != 'neautral']
df['len_tokens'] = df['review'].progress_apply(lambda x: len(tokenizer.tokenize(x)))
df = df[df['len_tokens'] < 511]

100%|██████████| 60000/60000 [00:13<00:00, 4544.73it/s]


In [5]:
preproc =  pd.DataFrame()
preproc["tokenized"] = df['review'].progress_apply(lambda x: tokenizer(x, return_tensors='pt').to(device))


100%|██████████| 59996/59996 [00:33<00:00, 1791.43it/s]


In [6]:
with torch.no_grad():
    preproc["vectors"]   = preproc["tokenized"].progress_apply(lambda x: model(**x)[0][0][0])

 18%|█▊        | 10822/59996 [01:17<06:10, 132.55it/s]